In [5]:
import pandas as pd
import re
from sqlalchemy import create_engine
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score
import matplotlib.pyplot as plt
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import GridSearchCV
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, Input
from keras.layers.embeddings import Embedding

from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.initializers import RandomNormal

from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import log_loss
from keras.callbacks import ModelCheckpoint

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
! unzip /content/drive/MyDrive/All/Participants_Data_DCW.zip

Archive:  /content/drive/MyDrive/All/Participants_Data_DCW.zip
  inflating: test.csv                
  inflating: __MACOSX/._test.csv     
  inflating: train.csv               
  inflating: __MACOSX/._train.csv    
  inflating: submission.csv          
  inflating: __MACOSX/._submission.csv  


In [7]:
df = pd.read_csv('/content/train.csv')

In [8]:
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

In [9]:
def clean_text(text):
    text = text.lower()
    
    
    text = re.sub(r'@[A-Za-z0-9_]+','', text)
    
    # Dealing with URL links
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    text = re.sub(url_regex,'urlplaceholder', text)
    # A lot of url are write as follows: http bit.ly. Apply Regex for these cases
    utl_regex_2 = 'http [a-zA-Z]+\.[a-zA-Z]+'
    text = re.sub(utl_regex_2,'urlplaceholder', text)
    # Other formats: http : //t.co/ihW64e8Z
    utl_regex_3 = 'http \: //[a-zA-Z]\.(co|com|pt|ly)/[A-Za-z0-9_]+'
    text = re.sub(utl_regex_3,'urlplaceholder', text)
    
   
    
    
    # Contractions
    text = re.sub(r"what's", 'what is ', text)
    text = re.sub(r"can't", 'cannot', text)
    text = re.sub(r"\'s",' ', text)
    text = re.sub(r"\'ve", ' have ', text)
    text = re.sub(r"n't", ' not ', text)
    text = re.sub(r"im", 'i am ', text)
    text = re.sub(r"i'm", 'i am ', text)
    text = re.sub(r"\'re", ' are ', text)
    text = re.sub(r"\'d", ' would ', text)
    text = re.sub(r"\'ll", ' will ', text)
                  
    # Operations and special words  
    text = re.sub('#',' ', text)         
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub('foof', 'food', text)
    text = re.sub('msg', 'message', text)
    text = re.sub(' u ', 'you', text)
    
    # Ponctuation Removal
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    
    text = text.split()
    stop_words = stopwords.words("english")
    text = [tok for tok in text if tok not in stop_words]
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(w) for w in text]
    return ' '.join(text)

In [10]:
# Cleaning Text
df['Review'] = df['Review'].map(lambda x: clean_text(x))
X = df['Review']

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=10,ngram_range=(1,4), max_features=300)
vectorizer.fit(X) # fit has to happen only on train data
X = vectorizer.transform(X)
X.shape

(6136, 300)

In [12]:
X = X.todense()

In [13]:
'''
vocabulary_size = 20000
tokenizer = Tokenizer(num_words=vocabulary_size)
tokenizer.fit_on_texts(df['Review'])
sequences = tokenizer.texts_to_sequences(df['Review'])
'''

"\nvocabulary_size = 20000\ntokenizer = Tokenizer(num_words=vocabulary_size)\ntokenizer.fit_on_texts(df['Review'])\nsequences = tokenizer.texts_to_sequences(df['Review'])\n"

In [14]:
'''
MAXLEN = 50
X = pad_sequences(sequences, maxlen=MAXLEN)
type(X)
'''

'\nMAXLEN = 50\nX = pad_sequences(sequences, maxlen=MAXLEN)\ntype(X)\n'

In [15]:
y = np.array(df[[ 'Components', 'Delivery and Customer Support',
       'Design and Aesthetics', 'Dimensions', 'Features', 'Functionality',
       'Installation', 'Material', 'Price', 'Quality', 'Usability','Polarity']])



In [16]:
def get_model(n_inputs, n_outputs):
  batch_size = 256 # It is the sample size of inputs to be processed at each training stage. 
  hidden_units = 128
  dropout = 0.45

  # Nossa  MLP com ReLU e Dropout 
  model = Sequential()

  model.add(Dense(hidden_units, input_dim=n_inputs,activation='relu',
          kernel_initializer='he_uniform'))

  model.add(Dropout(dropout))


  model.add(Dense(64,activation='relu',
          kernel_initializer='he_uniform'))

  model.add(Dropout(dropout))

  model.add(Dense(n_outputs))
 
  model.add(Activation('sigmoid'))

  model.compile(loss='binary_crossentropy', optimizer='adam')
  
  return model

In [17]:
def evaluate_model(X,y):
  results_test = []
  results_train =[]
  callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3,min_delta = 0.01)
  n_inputs, n_outputs = X.shape[1], y.shape[1]
  cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)
  for train_ix,test_ix in cv.split(X):
    X_train, X_test = X[train_ix], X[test_ix]
    y_train,y_test = y[train_ix],y[test_ix]
    
    model = get_model(n_inputs, n_outputs)
    
    model.fit(X_train,y_train,epochs = 10,callbacks = callback)
    yhat_train = model.predict(X_train)
    yhat_test = model.predict(X_test)

    train_log_loss = log_loss(y_train, yhat_train)
    test_log_loss = log_loss(y_test,yhat_test)
    
    results_train.append(train_log_loss)
    results_test.append(test_log_loss)

  return results_train,results_test,model

In [18]:
results_train,results_test,model = evaluate_model(X, y)
print(results_train)
print(results_test)

Epoch 1/10
154/154 [==============================] - 1s 2ms/step - loss: 0.4507
Epoch 2/10
154/154 [==============================] - 0s 2ms/step - loss: 0.3417
Epoch 3/10
154/154 [==============================] - 0s 2ms/step - loss: 0.2956
Epoch 4/10
154/154 [==============================] - 0s 2ms/step - loss: 0.2687
Epoch 5/10
154/154 [==============================] - 0s 2ms/step - loss: 0.2543
Epoch 6/10
154/154 [==============================] - 0s 2ms/step - loss: 0.2452
Epoch 7/10
154/154 [==============================] - 0s 2ms/step - loss: 0.2350
Epoch 8/10
154/154 [==============================] - 0s 2ms/step - loss: 0.2322
Epoch 9/10
154/154 [==============================] - 0s 2ms/step - loss: 0.2283
Epoch 10/10
154/154 [==============================] - 0s 2ms/step - loss: 0.2214
Epoch 1/10
154/154 [==============================] - 1s 2ms/step - loss: 0.4473
Epoch 2/10
154/154 [==============================] - 0s 2ms/step - loss: 0.3396
Epoch 3/10
154/154 [=======